In [93]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import missingno


ModuleNotFoundError: No module named 'missingno'

In [16]:
NSCH19 = pd.read_sas("data/nsch_2019e_topical.sas7bdat")
#NSCH22 = pd.read_sas("data/nsch_2022e_topical.sas7bdat")

In [18]:
NSCH19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29433 entries, 0 to 29432
Columns: 448 entries, FIPSST to HOUSE_GEN
dtypes: float64(444), object(4)
memory usage: 100.6+ MB


In [23]:
geo_features = ['FIPSST']
## K7Q02R_R is number of days missed in school because of illness or injury.
## Responses are binned in intervals of 3 days, with 1 meaning no missed days, 5 meaning 11+ days, 6 meaning no enrolled
## in school
edu_features = ['K7Q02R_R']
med_features = ['AVAILABLE', 'APPOINTMENT', 'ISSUECOST', 
                'NOTELIG', 'NOTOPEN', 'TRANSPORTCC',
                'CURRCOV', 
                'HOWMUCH', #how much was paid for health care
                'INSTYPE', #public/ priv/ none
                'K3Q04_R', #any type of health ins
                'K12Q12', #govt assistance plan (yes/ no)
                'K3Q21B', 'K3Q20', 'K3Q25', #cost reasonable, needs covered, allowed to see providers
                'K4Q01', #is there place can take child when they are sick
                'K4Q24_R', 'K4Q26', #how difficult to get specialist care
                'K4Q27', #Was there any time when child needed health care but did not receive?
                'S4Q01', #did this child see a doctor
                'K5Q31_R', 'K5Q32' #satisfactory communication from doctor to school
                ]           

features = geo_features + edu_features + med_features

In [87]:
def FIPS_to_State(data, state='both'):
    #This function specifically includes a column corresponding
    #to either the state abbreviation if state='abbr' or
    #the full state name if state='full' or both='both'.  Note that this is specifically written
    #for NSCH data which names their FIPS column FIPSST.  This also requires the
    #FIPS_State.csv file which translates between FIPS and State.  

    #Also note: this function changes the FIP code to an integer from a byte string.
    #I c

    FIPS_state = pd.read_csv('data/FIPS_State.csv')
    FIPS_state['FIPSST'] = FIPS_state.FIPS

    data.FIPSST = data.FIPSST.apply(int)

    if state == 'abbr':
        data = data.merge(FIPS_state[['FIPSST', 'Abbreviation']], on='FIPSST')

    if state == 'full':
        data = data.merge(FIPS_state[['FIPSST', 'Name']], on='FIPSST') 

    if state == 'both':
        data = data.merge(FIPS_state[['FIPSST', 'Name', 'Abbreviation']], on='FIPSST')                

    return data
